<a href="https://colab.research.google.com/github/datajcthemax/Dynamic-Indices-cluster/blob/jai/fetch_balance_sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import pandas as pd
import requests

# 데이터를 가져올 URL
url = "https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=AAPL&apikey=QA7AWI9EFI267BNJ"

# requests를 사용하여 URL에서 데이터를 가져옴
response = requests.get(url)
data = response.json()

# 알파밴티지 API 응답에서 'annualReports' 키를 사용하여 연간 재무제표 데이터를 추출
# (주: 'annualReports' 또는 다른 키 이름이 변경될 수 있으므로 실제 응답 구조를 확인하십시오.)
annual_reports = data['annualReports']

# 데이터를 pandas DataFrame으로 변환
df = pd.DataFrame(annual_reports)

print(df.head())  # 처음 5개의 데이터 행을 출력


  fiscalDateEnding reportedCurrency   totalAssets totalCurrentAssets  \
0       2022-09-30              USD  352755000000       135405000000   
1       2021-09-30              USD  351002000000       134836000000   
2       2020-09-30              USD  323888000000       143713000000   
3       2019-09-30              USD  338516000000       162819000000   
4       2018-09-30              USD  365725000000       131339000000   

  cashAndCashEquivalentsAtCarryingValue cashAndShortTermInvestments  \
0                           23646000000                 48304000000   
1                           34940000000                 62639000000   
2                           38016000000                 90943000000   
3                           48844000000                100557000000   
4                           25913000000                 25913000000   

    inventory currentNetReceivables totalNonCurrentAssets  \
0  4946000000           60932000000          217350000000   
1  6580000000     

In [3]:
# DataFrame을 CSV 파일로 저장
df.to_csv("apple_balance_sheet.csv", index=False)  # index=False는 인덱스 정보를 CSV에 포함시키지 않도록 합니다.

In [5]:
# 앞선 코드의 연장...

# DataFrame을 전치
transposed_df = df.transpose()

# 전치된 DataFrame의 첫 번째 행이 년도이므로 이를 칼럼으로 설정
transposed_df.columns = transposed_df.iloc[0]
transposed_df = transposed_df.drop(transposed_df.index[0])  # 년도를 칼럼으로 설정한 후, 원래의 년도 행을 삭제

# CSV 파일로 저장
transposed_df.to_csv("transposed_apple_balance_sheet.csv")


In [1]:
pip install yfinance pandas


In [ ]:
import yfinance as yf
import pandas as pd

# NASDAQ 상장 회사 목록 가져오기
nasdaq_listed_companies = pd.read_csv('http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt', sep='|')
symbols = nasdaq_listed_companies['Symbol'].dropna().tolist()

# 모든 회사의 재무제표를 저장할 빈 DataFrame 생성
all_balance_sheets = pd.DataFrame()

for symbol in symbols:
    try:
        ticker = yf.Ticker(symbol)
        balance_sheet = ticker.balance_sheet

        # DataFrame이 비어 있지 않을 경우에만 데이터 처리
        if not balance_sheet.empty:
            balance_sheet_transposed = balance_sheet.transpose()
            balance_sheet_transposed['Symbol'] = symbol
            all_balance_sheets = pd.concat([all_balance_sheets, balance_sheet_transposed])
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        continue

all_balance_sheets.to_csv("all_nasdaq_balance_sheets_yfinance.csv")
